In [0]:
flight_schema_ddl = """FL_DATE STRING, OP_CARRIER STRING, OP_CARRIER_FL_NUM INT, ORIGIN STRING, ORIGIN_CITY_NAME STRING, DEST STRING, DEST_CITY_NAME STRING, CRS_DEP_TIME INT, DEP_TIME INT, WHEELS_ON INT, TAXI_IN INT, CRS_ARR_TIME INT, ARR_TIME INT, CANCELLED STRING, DISTANCE INT"""


flight_time_raw_df = spark.read\
                    .format("json")\
                      .schema(flight_schema_ddl)\
                          .option("mode", "FAILFAST")\
                              .option("dateFormat", "M/d/y")\
                                .load("/FileStore/tables/flight_time.json")

flight_time_raw_df.show(10)

+--------+----------+-----------------+------+----------------+----+--------------+------------+--------+---------+-------+------------+--------+---------+--------+
| FL_DATE|OP_CARRIER|OP_CARRIER_FL_NUM|ORIGIN|ORIGIN_CITY_NAME|DEST|DEST_CITY_NAME|CRS_DEP_TIME|DEP_TIME|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|CANCELLED|DISTANCE|
+--------+----------+-----------------+------+----------------+----+--------------+------------+--------+---------+-------+------------+--------+---------+--------+
|1/1/2000|        DL|             1451|   BOS|      Boston, MA| ATL|   Atlanta, GA|        1115|    1113|     1343|      5|        1400|    1348|        0|     946|
|1/1/2000|        DL|             1479|   BOS|      Boston, MA| ATL|   Atlanta, GA|        1315|    1311|     1536|      7|        1559|    1543|        0|     946|
|1/1/2000|        DL|             1857|   BOS|      Boston, MA| ATL|   Atlanta, GA|        1415|    1414|     1642|      9|        1721|    1651|        0|     946|
|1/1/2000|

In [0]:
flight_time_raw_df.printSchema()

root
 |-- FL_DATE: string (nullable = true)
 |-- OP_CARRIER: string (nullable = true)
 |-- OP_CARRIER_FL_NUM: integer (nullable = true)
 |-- ORIGIN: string (nullable = true)
 |-- ORIGIN_CITY_NAME: string (nullable = true)
 |-- DEST: string (nullable = true)
 |-- DEST_CITY_NAME: string (nullable = true)
 |-- CRS_DEP_TIME: integer (nullable = true)
 |-- DEP_TIME: integer (nullable = true)
 |-- WHEELS_ON: integer (nullable = true)
 |-- TAXI_IN: integer (nullable = true)
 |-- CRS_ARR_TIME: integer (nullable = true)
 |-- ARR_TIME: integer (nullable = true)
 |-- CANCELLED: string (nullable = true)
 |-- DISTANCE: integer (nullable = true)



In [0]:
flight_time_raw_df.show(10)

+--------+----------+-----------------+------+----------------+----+--------------+------------+--------+---------+-------+------------+--------+---------+--------+
| FL_DATE|OP_CARRIER|OP_CARRIER_FL_NUM|ORIGIN|ORIGIN_CITY_NAME|DEST|DEST_CITY_NAME|CRS_DEP_TIME|DEP_TIME|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|CANCELLED|DISTANCE|
+--------+----------+-----------------+------+----------------+----+--------------+------------+--------+---------+-------+------------+--------+---------+--------+
|1/1/2000|        DL|             1451|   BOS|      Boston, MA| ATL|   Atlanta, GA|        1115|    1113|     1343|      5|        1400|    1348|        0|     946|
|1/1/2000|        DL|             1479|   BOS|      Boston, MA| ATL|   Atlanta, GA|        1315|    1311|     1536|      7|        1559|    1543|        0|     946|
|1/1/2000|        DL|             1857|   BOS|      Boston, MA| ATL|   Atlanta, GA|        1415|    1414|     1642|      9|        1721|    1651|        0|     946|
|1/1/2000|

In [0]:
import pyspark.sql.functions as F

flight_time_raw_df1 = flight_time_raw_df\
                            .withColumn("FL_DATE_DT", F.to_timestamp(F.col("FL_DATE"), "yyyy-mm-dd"))

flight_time_raw_df1.printSchema()

root
 |-- FL_DATE: string (nullable = true)
 |-- OP_CARRIER: string (nullable = true)
 |-- OP_CARRIER_FL_NUM: integer (nullable = true)
 |-- ORIGIN: string (nullable = true)
 |-- ORIGIN_CITY_NAME: string (nullable = true)
 |-- DEST: string (nullable = true)
 |-- DEST_CITY_NAME: string (nullable = true)
 |-- CRS_DEP_TIME: integer (nullable = true)
 |-- DEP_TIME: integer (nullable = true)
 |-- WHEELS_ON: integer (nullable = true)
 |-- TAXI_IN: integer (nullable = true)
 |-- CRS_ARR_TIME: integer (nullable = true)
 |-- ARR_TIME: integer (nullable = true)
 |-- CANCELLED: string (nullable = true)
 |-- DISTANCE: integer (nullable = true)
 |-- FL_DATE_DT: timestamp (nullable = true)



In [0]:
from pyspark.sql.functions import to_date, expr

flight_time_df = flight_time_raw_df\
        .withColumn("FL_DATE", to_date("FL_DATE", "M/d/y"))\
            .withColumn("CANCELLED", expr("if(CANCELLED == 1, true, false)"))

In [0]:
flight_time_df.show(10)

+----------+----------+-----------------+------+----------------+----+--------------+------------+--------+---------+-------+------------+--------+---------+--------+
|   FL_DATE|OP_CARRIER|OP_CARRIER_FL_NUM|ORIGIN|ORIGIN_CITY_NAME|DEST|DEST_CITY_NAME|CRS_DEP_TIME|DEP_TIME|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|CANCELLED|DISTANCE|
+----------+----------+-----------------+------+----------------+----+--------------+------------+--------+---------+-------+------------+--------+---------+--------+
|2000-01-01|        DL|             1451|   BOS|      Boston, MA| ATL|   Atlanta, GA|        1115|    1113|     1343|      5|        1400|    1348|    false|     946|
|2000-01-01|        DL|             1479|   BOS|      Boston, MA| ATL|   Atlanta, GA|        1315|    1311|     1536|      7|        1559|    1543|    false|     946|
|2000-01-01|        DL|             1857|   BOS|      Boston, MA| ATL|   Atlanta, GA|        1415|    1414|     1642|      9|        1721|    1651|    false|     946

In [0]:
flight_time_df.write\
    .format("parquet")\
        .mode("overwrite")\
            .saveAsTable("flight_time_tbl")


1. write options: overwrite, append, error or errorifexists, ignore
2. Spark tables dont impose any PK constraints
3. Append and overwrite modes also create the table if it doesn't exist.
4. Ignore if the table is already there. If doesnt exist then write.

In [0]:
%sql

describe extended flight_time_tbl

col_name,data_type,comment
FL_DATE,date,null
OP_CARRIER,string,null
OP_CARRIER_FL_NUM,int,null
ORIGIN,string,null
ORIGIN_CITY_NAME,string,null
DEST,string,null
DEST_CITY_NAME,string,null
CRS_DEP_TIME,int,null
DEP_TIME,int,null
WHEELS_ON,int,null



# Inspecting DW location

In [0]:
%fs ls /user/hive/warehouse/flight_time_tbl

path,name,size,modificationTime
dbfs:/user/hive/warehouse/flight_time_tbl/_SUCCESS,_SUCCESS,0,1749977683000
dbfs:/user/hive/warehouse/flight_time_tbl/_committed_2415843550852993408,_committed_2415843550852993408,816,1749977683000
dbfs:/user/hive/warehouse/flight_time_tbl/_started_2415843550852993408,_started_2415843550852993408,0,1749977678000
dbfs:/user/hive/warehouse/flight_time_tbl/part-00000-tid-2415843550852993408-320680e5-fcd3-47b8-94da-f1300d7c7c5b-76-1-c000.snappy.parquet,part-00000-tid-2415843550852993408-320680e5-fcd3-47b8-94da-f1300d7c7c5b-76-1-c000.snappy.parquet,501013,1749977683000
dbfs:/user/hive/warehouse/flight_time_tbl/part-00001-tid-2415843550852993408-320680e5-fcd3-47b8-94da-f1300d7c7c5b-77-1-c000.snappy.parquet,part-00001-tid-2415843550852993408-320680e5-fcd3-47b8-94da-f1300d7c7c5b-77-1-c000.snappy.parquet,492268,1749977683000
dbfs:/user/hive/warehouse/flight_time_tbl/part-00002-tid-2415843550852993408-320680e5-fcd3-47b8-94da-f1300d7c7c5b-78-1-c000.snappy.parquet,part-00002-tid-2415843550852993408-320680e5-fcd3-47b8-94da-f1300d7c7c5b-78-1-c000.snappy.parquet,485042,1749977682000
dbfs:/user/hive/warehouse/flight_time_tbl/part-00003-tid-2415843550852993408-320680e5-fcd3-47b8-94da-f1300d7c7c5b-79-1-c000.snappy.parquet,part-00003-tid-2415843550852993408-320680e5-fcd3-47b8-94da-f1300d7c7c5b-79-1-c000.snappy.parquet,488176,1749977682000
dbfs:/user/hive/warehouse/flight_time_tbl/part-00004-tid-2415843550852993408-320680e5-fcd3-47b8-94da-f1300d7c7c5b-80-1-c000.snappy.parquet,part-00004-tid-2415843550852993408-320680e5-fcd3-47b8-94da-f1300d7c7c5b-80-1-c000.snappy.parquet,481072,1749977683000
dbfs:/user/hive/warehouse/flight_time_tbl/part-00005-tid-2415843550852993408-320680e5-fcd3-47b8-94da-f1300d7c7c5b-81-1-c000.snappy.parquet,part-00005-tid-2415843550852993408-320680e5-fcd3-47b8-94da-f1300d7c7c5b-81-1-c000.snappy.parquet,498348,1749977682000
dbfs:/user/hive/warehouse/flight_time_tbl/part-00006-tid-2415843550852993408-320680e5-fcd3-47b8-94da-f1300d7c7c5b-82-1-c000.snappy.parquet,part-00006-tid-2415843550852993408-320680e5-fcd3-47b8-94da-f1300d7c7c5b-82-1-c000.snappy.parquet,497461,1749977682000



# Query the metadata store


## 1. SQL Commands

In [0]:
%sql

show tables

database,tableName,isTemporary
default,flight_time_tbl,false



# 2. Spark Catalog API

In [0]:
spark.catalog.listTables()

Out[13]: [Table(name='flight_time_tbl', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False)]


# External Tables


Use flight_time_tbl as external location. We'll create external table on top of flight_time_tbl

In [0]:
%sql 

-- managed table
create table if not exists managed_flight_time_tbl like flight_time_tbl
;

-- external table
create table if not exists ext_flight_time_tbl like flight_time_tbl
location "/user/hive/warehouse/flight_time_tbl"
;

-- like command copies schema from existing table
-- external table should be created where the original table stores its data.

In [0]:
%sql

describe extended ext_flight_time_tbl

col_name,data_type,comment
FL_DATE,date,null
OP_CARRIER,string,null
OP_CARRIER_FL_NUM,int,null
ORIGIN,string,null
ORIGIN_CITY_NAME,string,null
DEST,string,null
DEST_CITY_NAME,string,null
CRS_DEP_TIME,int,null
DEP_TIME,int,null
WHEELS_ON,int,null


In [0]:
spark.catalog.listTables()

Out[17]: [Table(name='ext_flight_time_tbl', catalog='spark_catalog', namespace=['default'], description=None, tableType='EXTERNAL', isTemporary=False),
 Table(name='flight_time_tbl', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='managed_flight_time_tbl', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False)]